In [ ]:
# 0. INSTALL REQUIRED LIBRARIES
!pip install pandas prince pyreadstat matplotlib seaborn openpyxl

# To run categorical association tests
!pip install scipy researchpy


# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from scipy.stats import chi2_contingency
from collections import Counter

# 2. LOAD DATA (SAV + METADATA)
df, meta = pyreadstat.read_sav(r"C:")  # data under embargo 

# CHECK BASIC INFO
print(df.shape)
print(df.dtypes.value_counts())
print(df.head())

In [ ]:
# EXCLUDE UNNECESSARY VAR
# Check variables to know what to remove
print(df.columns.tolist())

In [ ]:
# CHECK UNIQUE VALUES FOR EACH VAR
for col in df.columns:
    print(f'{col.upper()} --> valores únicos:', df[col].unique())

In [ ]:
# Exclude unnecessary var for the analysis (D10-D13 are var depending on D9, thus few responses)
df.drop(["status", "lastpage", "startlanguage", "startdate", "datestamp", "ipaddr", 'codcom_1', 'codcom_2', 'ampiezza','TITOLOSTUDIORECODE', "Area", "provincia", "ampiezzaclasse", "D09s", "D10", "D10s", "D11", "D11s", "D12", "D12s", "peso"], axis=1, inplace=True)
print(df.columns)  # Show variables remaining after removing unnecessary ones 

In [ ]:
print("After var dropping, the dataset comprised observations and variables:", df.shape)

In [ ]:
df.dtypes

In [ ]:
# EXTRACT THE YEAR FORM THE DATE "submitdate" VARIABLE
# Check "submitdate" type
type(df["submitdate"])

- The survey was implemented/submited in 2025:

In [ ]:
# Extract survey year from submitdate
df["year_extracted"] = pd.to_datetime(df["submitdate"], errors="coerce").dt.year

# Check the distinct years found in the dataset
df["year_extracted"].unique()


In [ ]:
# After defining the var "year_extracted", var "submitdate" becomes unnecessary, thus it is excluded
df.drop(["submitdate"], axis=1, inplace=True)
print(df.columns)  # Show variables remaining after removing unnecessary ones

In [ ]:
# CALCULATE PARTICIPANTS' AGE
# Calculate the age of participants as difference between year of survey implementation and birth year(ETA)
df['age'] = 2025 - df['ETA']
df["age"].unique()

- There are no outliers 

In [ ]:
# DEFINE CATEGORICAL VAR WITH A LIST
categorical_vars = [
    "GEN", "RETA", "regione", 'D01', 'D02',
       'D03', 'D04', 'D05', 'D06_1', 'D07', 'D08', 'D09', 'D13', 'D14', 'D15',
       'D16', 'D17', 'D18', 'D19', 'D20', 'D22'
]

for col in categorical_vars:
    print(col, "→", df[col].unique())

- There are no missing values:

In [ ]:
# MISSING VALUES OVERVIEW
missing = pd.DataFrame({
    "Missing": df[categorical_vars].isna().sum(),
    "Percent": df[categorical_vars].isna().mean() * 100
})
print("\nMissing values:")
print(missing)

In [ ]:
# Check for null values in each column
# Returns True where values are null
df.isna().any()

In [ ]:
# BUILD VALUE LABEL DICTIONARY FROM METADATA

variable_to_labels = {}

for var, label_key in meta.variable_to_label.items():
    # Store label dict if available in metadata
    if label_key in meta.value_labels:
        variable_to_labels[var] = meta.value_labels[label_key]

# Optional: Quick inspection
import pprint
print("\n Labels successfully mapped for these variables:\n")
pprint.pprint(list(variable_to_labels.keys()))

In [ ]:
# Use a dict to connect each variable name to its descriptive label (metadata)
# Two lists from metadata: meta.column_names → list of variable names, meta.column_labels → list of descriptive labels;
# Pair each name with its label: zip()
# Convert pairs into a usable dictionary: dict()
variable_names = dict(zip(meta.column_names, meta.column_labels))
variable_names["D19"]

- VALUE CO-OCCURRENCE
- Used to identify common response patterns across variables. Only two observations share the same pattern, indicating high heterogeneity in response behaviors:

In [ ]:
# VALUE CO-OCCURRENCE
# Example: most common response patterns across selected variables
patterns = df[categorical_vars].astype(str).agg("-".join, axis=1)  # axis=1 → apply the function row-wise (across columns)
# "-".join → the function being applied: joins all values in the row into a single string separated by -
# agg (short for aggregate) is a method used to apply one or more functions across rows or columns

print("\nMost common response combinations:")
print(Counter(patterns).most_common(10))

# Contar combinaciones
combo_counts = df[categorical_vars].value_counts()

# Mostrar los 10 más frecuentes
combo_counts.head(5)

- UNIVARIATE ANALSYS OF AGE
- The mean and median are very similar, suggesting that the distribution of age values is approximately symmetric. This is further confirmed by the skewness statistic (a measure of how asymmetrically a distribution is shaped around its mean) of -0.096, which indicates that the distribution is nearly symmetric, with only a negligible slight left-tail tendency:

In [ ]:
# Summary for numerical variables
summary = df['age'].describe()
print(summary)

In [ ]:
# skewness_value = ((age - mean)**3).mean() / (std**3)
# Skewness > 0 → right-skewed (long tail to the right)
# Skewness < 0 → left-skewed (long tail to the left)
# Skewness ≈ 0 → symmetric distribution
skewness_value = df['age'].skew()
print("Skewness:", skewness_value)

- Histogram showong the raw counts of age values:

In [ ]:
# VISUALISATION FOR CONTINUOUS VAR "age": HISTOGRAM
sns.histplot(df['age'], bins=100)
# Outliers: there are no outliers for var "age"

- The plot shows a histogram of age (density), overlaid with a KDE line representing the estimated distribution, a Gaussian curve fitted to the data, and a dashed line indicating the mean age
- This visualization improves on a simple histogram by showing the data’s shape and smooth trends, allowing comparison with a theoretical distribution, and clearly indicating the mean:

In [ ]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
from scipy.stats import norm

age = df['age']

# Plot histogram
sns.histplot(age, bins=100, stat='density', color='skyblue', edgecolor='black', alpha=0.5)

# Overlay KDE (estimated distribution)
sns.kdeplot(age, color='orange', linewidth=2, label='Estimated distribution (KDE)')

# Overlay Gaussian curve for comparison
mu, std = norm.fit(age)
x = np.linspace(age.min(), age.max(), 100)
plt.plot(x, norm.pdf(x, mu, std), 'r--', linewidth=2, label='Gaussian fit')

# Add mean line
plt.axvline(age.mean(), color='green', linestyle='dashed', linewidth=2, label=f'Mean: {age.mean():.2f}')

# Labels and legend
plt.xlabel('Age')
plt.ylabel('Density')
plt.title('Histogram with KDE and Gaussian Fit')
plt.legend()
plt.show()


- Kolmogorov–Smirnov test for normality on "age"
- p ≤ 0.05 → the distribution of age significantly differs from a normal distribution
- the H0 of normal distribution is rejected:

In [ ]:
from labels import translated_labels, variable_labels, map_codes, translate
from scipy.stats import kstest, norm
# import numpy as np

# Extract the age var
age = df["age"]

# Standardize age to mean 0, std 1 (K-S test requires this when testing normality)
age_standardized = (age - np.mean(age)) / np.std(age, ddof=1)

# Perform K-S test for normality
ks_stat, p_value = kstest(age_standardized, 'norm')

print("Kolmogorov–Smirnov Test for Normality (Age)")
print(f"KS Statistic: {ks_stat:.4f}")
print(f"P-value: {p_value:.4f}")


In [ ]:
# UNIVARIATE ANALYSIS FOR CATEGORICAL VARIABLES

# # labels.py — FULL TRANSLATION DICTIONARY + HELPERS
from labels import translated_labels, variable_labels, map_codes, translate


def categorical_summary(df, categorical_vars):
    """
    Prints frequency tables with counts, percentages, and labels
    for a list of categorical variables.
    
    Automatically uses translated_labels and variable_labels
    from labels.py.
    """

    summary_tables = {}

    for col in categorical_vars:
        # Calculate frequency and percentage
        freq = df[col].value_counts(dropna=False).sort_index()
        percent = df[col].value_counts(dropna=False, normalize=True).sort_index() * 100

        # Get variable descriptive label
        var_label = variable_labels.get(col, col)

        # Print header
        print("\n" + "="*60)
        print(f"🔹 Variable: {col}  →  {var_label}")
        print("="*60)

        # Map numeric codes to labels using map_codes
        labels_series = map_codes(freq.index.to_series(), col)

        # Build summary table
        summary = pd.DataFrame({
            "Category": freq.index,
            "Label": labels_series.values,
            "Count": freq.values,
            "Percent (%)": percent.round(2)
        })

        summary_tables[col] = summary
        print(summary.to_string(index=False))

    return summary_tables


# Run the summary
tables = categorical_summary(df, categorical_vars)


In [ ]:
# VISUALIZATION TECHNIQUE OF % DISTRIBUTION OF SEX VAR - PIE CHART

# #import matplotlib.pyplot as plt
#import pandas as pd

from labels import translated_labels, variable_labels, map_codes, translate

# Prepare data for 'GEN'
# df['GEN'] contains numeric codes (1.0, 2.0, etc.)
gen_labels = map_codes(df['GEN'], 'GEN')  # Convert codes to text labels
sex_counts = gen_labels.value_counts()    # Count occurrences

# Pie chart
plt.figure(figsize=(6,6))
plt.pie(
    sex_counts.values,
    labels=sex_counts.index,          # Use translated labels
    autopct='%1.1f%%',
    startangle=90,
    colors=['skyblue', 'lightpink', 'lightgray'],  # add color for 'Prefer not to answer' if exists
    textprops={'fontsize': 14, 'weight': 'bold'}  # Increase font size and make bold
)
plt.title('Sex Distribution (%)', fontsize=16, weight='bold')  # bold title
plt.axis('equal')  # Equal aspect ratio ensures the pie is circular
plt.show()

In [ ]:
# UNIVARIATE ANALYSIS FOR CATEGORICAL VAR - PLOT DISTRIBUTIONS (bar charts)

#import matplotlib.pyplot as plt
#import seaborn as sns

from labels import translated_labels, variable_labels, map_codes, translate

def plot_categorical_var(df, categorical_vars, palette="Set2"):
    """
    Plot bar charts for categorical variables with value labels and percentages.
    Automatically uses translated_labels and variable_labels from labels.py.
    Works for both numeric codes and already text columns.
    """

    FIG_SIZE = (8, 5)

    for col in categorical_vars:
        # Get variable descriptive label
        var_name = variable_labels.get(col, col)

        # Get value labels dictionary
        labels_dict = translated_labels.get(col, {})
        # Convert numeric keys to float
        labels_dict = {float(k): v for k, v in labels_dict.items()}

        plt.figure(figsize=FIG_SIZE)
        ax = sns.countplot(x=df[col], palette=palette)

        # Replace x ticks with category labels safely
        new_labels = []
        for x in ax.get_xticklabels():
            text = x.get_text()
            try:
                # Try to map as float
                new_labels.append(labels_dict.get(float(text), text))
            except ValueError:
                # Keep original text if not numeric
                new_labels.append(text)

        ax.set_xticklabels(new_labels, rotation=40, ha="right")

        # Add percentages on bars
        total = len(df[col])
        for p in ax.patches:
            height = p.get_height()
            if height > 0:
                percent = f"{(height / total) * 100:.1f}%"
                ax.text(
                    p.get_x() + p.get_width() / 2,
                    height,
                    percent,
                    ha="center",
                    va="bottom",
                    fontsize=10
                )

        # Titles and layout
        plt.title(f"Distribution of {col} – {var_name}", fontsize=14, weight='bold')
        plt.ylabel("Count")
        plt.xlabel("")
        plt.tight_layout()
        plt.show()

# Usage example
plot_categorical_var(df, categorical_vars, palette="Pastel1")

- According to the Chi Squared test, four variables are associated to the sex var:

In [ ]:
# BIVARIATE ANALYSIS: chi2 TEST FOR INDEPENDENCE TO ASSESS POSSIBLE ASSOCIATION BETWEEN SEX AND THE OTHER VARS

# import pandas as pd
from scipy.stats import chi2_contingency
from labels import translated_labels, variable_labels, map_codes, translate

def chi2_summary_table(df, target_var, categorical_vars, alpha=0.05, round_digits=3, translate_values=True):
    """
    Compute Chi² test for one target variable against a list of categorical variables.

    Parameters
    ----------
    df : pd.DataFrame
    target_var : str
    categorical_vars : list
    alpha : float
    round_digits : int
    translate_values : bool

    Returns
    -------
    pd.DataFrame
        Summary table with Chi², p-value, degrees of freedom, significance, 
        and fully translated contingency tables.
    """
    # ✅ Ensure full text display in Jupyter
    pd.set_option('display.max_colwidth', None)  # shows full text in DataFrame cells

    summary = []

    for var in categorical_vars:
        if var == target_var:
            continue

        subset = df[[target_var, var]].dropna()

        # Numeric contingency table for Chi²
        contingency_table_numeric = pd.crosstab(subset[target_var], subset[var])

        if contingency_table_numeric.shape[0] < 2 or contingency_table_numeric.shape[1] < 2:
            continue

        chi2_stat, p_value, dof, _ = chi2_contingency(contingency_table_numeric)

        var_name = variable_labels.get(var, var)
        sig = "YES" if p_value < alpha else "NO"

        # Optional translated contingency table
        if translate_values:
            subset_translated = translate(subset)
            contingency_table_display = pd.crosstab(subset_translated[target_var],
                                                    subset_translated[var])
        else:
            contingency_table_display = contingency_table_numeric

        contingency_dict = contingency_table_display.to_dict()

        summary.append({
            "Variable": var,
            "Description": var_name,
            "Chi2": round(chi2_stat, round_digits),
            "p-value": round(p_value, round_digits),
            "dof": dof,
            "Significant": sig,
            "Contingency": contingency_dict
        })

    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="p-value").reset_index(drop=True)
    return summary_df


# Example usage in Jupyter:
categorical_vars = [
    "GEN", "RETA", "regione", 'D01', 'D02',
    'D03', 'D04', 'D05', 'D06_1', 'D07', 'D08', 'D09', 'D13', 'D14', 'D15',
    'D16', 'D17', 'D18', 'D19', 'D20', 'D22'
]

chi2_results = chi2_summary_table(df, target_var="GEN", categorical_vars=categorical_vars)

from IPython.display import display
display(chi2_results)


In [ ]:
#  'D22': "Education level" by sex
# Counts table
ct_counts = pd.crosstab(df['D22'], df['GEN'])

# Percentages by row
ct_percent = pd.crosstab(df['D22'], df['GEN'], normalize='index') * 100

print("COUNTS:")
print(ct_counts)

print("\nPERCENTAGES (% by row):")
print(ct_percent.round(2))


- Of those who think climate change is not real, 67% are male, while 33% are female, indicating a higher proportion of males in this group:

In [ ]:
# BIVARIATE ANALYSIS: DISTRIBUTION OF SEX AMONG CATEGORIES OF THE FOUR VARIABLES THAT ARE ASSOCIATED TO SEX
# According to the Chi Squared test, four variables are associated to the sex var --> check those associations with plots
import matplotlib.pyplot as plt
from labels import translated_labels, variable_labels, map_codes, translate  # imports

# List of variables to plot
variables = ['D01', 'D13', 'D14', 'D22']

for var in variables:
    # Translate both the variable and GEN for readable labels
    df_plot = df[[var, 'GEN']].copy()
    df_plot[var] = map_codes(df_plot[var], var)
    df_plot['GEN'] = map_codes(df_plot['GEN'], 'GEN')

    # Group by variable and sex
    counts = df_plot.groupby([var, 'GEN']).size().unstack()

    # Calculate proportion in %
    counts_percent = counts.div(counts.sum(axis=1), axis=0) * 100

    # Plot stacked bar
    ax = counts_percent.plot(kind='bar', stacked=True, figsize=(8,5), colormap='tab20')

    # Add percentage labels inside each bar
    for i, row in enumerate(counts_percent.values):
        cum_sum = 0
        for j, val in enumerate(row):
            if val > 0:  # avoid labeling zero
                ax.text(
                    i,                # x position (bar index)
                    cum_sum + val/2,  # y position (middle of segment)
                    f"{val:.1f}%",    # label
                    ha='center', va='center', color='white', fontsize=9
                )
                cum_sum += val  # update cumulative height for next segment

    # Titles and layout using descriptive variable names from labels.py
    label = variable_labels.get(var, var)  # ✅ use variable_labels (full descriptive variable name from labels.py) instead of meta.column_labels
    plt.title(f"Distribution of Sex by {label}")
    plt.xlabel(label)
    plt.ylabel("Proportion (%)")
    plt.legend(title="Sex")
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()


- There are 12 variables that show a statistical significant association with the outcome var D05:

In [ ]:
# Chi-squared test for independence to test possible association between output var D05 and the other vars

# import pandas as pd
from scipy.stats import chi2_contingency
from labels import translated_labels, variable_labels, map_codes, translate  # Updated imports

def chi2_summary_table(df, target_var, categorical_vars, alpha=0.05, round_digits=3, translate_values=True):
    """
    Compute Chi² test for one target variable against a list of categorical variables.

    Parameters
    ----------
    df : pd.DataFrame
        Dataset containing categorical variables.
    target_var : str
        The categorical variable of interest (e.g., 'D05').
    categorical_vars : list
        List of categorical variables to test against the target_var.
    alpha : float
        Significance level for Chi² test.
    round_digits : int
        Decimal rounding for Chi² statistic and p-value.
    translate_values : bool
        Whether to translate numeric codes into readable labels using labels.py.

    Returns
    -------
    pd.DataFrame
        Summary table with Chi², p-value, degrees of freedom, significance, 
        and translated contingency tables.
    """

    summary = []

    for var in categorical_vars:
        if var == target_var:
            continue

        # Drop missing values in either column
        subset = df[[target_var, var]].dropna()

        # Optional translation for readable contingency table
        if translate_values:
            subset_translated = translate(subset)
        else:
            subset_translated = subset

        # Build contingency table (use translated values for readability)
        contingency_table = pd.crosstab(subset_translated[target_var],
                                        subset_translated[var])

        # Skip degenerate tables
        if contingency_table.shape[0] < 2 or contingency_table.shape[1] < 2:
            continue

        # Chi² is computed on numeric codes (subset) to avoid errors
        contingency_numeric = pd.crosstab(subset[target_var], subset[var])
        chi2_stat, p_value, dof, _ = chi2_contingency(contingency_numeric)

        # Use variable_labels from labels.py for descriptive names
        var_name = variable_labels.get(var, var)

        # Highlight significance
        sig = "YES" if p_value < alpha else "NO"

        # Optional: convert contingency table to dict for report-friendly format
        contingency_dict = contingency_table.to_dict()

        summary.append({
            "Variable": var,
            "Description": var_name,
            "Chi2": round(chi2_stat, round_digits),
            "p-value": round(p_value, round_digits),
            "dof": dof,
            "Significant": sig,
            "Contingency": contingency_dict
        })

    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="p-value").reset_index(drop=True)  # sorted by significance
    return summary_df


# Example usage:
categorical_vars = [
    "GEN", "RETA", "regione",'D01', 'D02',
    'D03', 'D04', 'D06_1', 'D07', 'D08', 'D09', 'D13', 'D14', 'D15',
    'D16', 'D17', 'D18', 'D19', 'D20', 'D22'
]

# Run Chi² summary for target variable 'D05' with labels integration
chi2_results = chi2_summary_table(df, target_var="D05", categorical_vars=categorical_vars)
display(chi2_results)


In [ ]:
# Filter significant variables
associated_vars = chi2_results[chi2_results['Significant'] == "YES"]

# Reset index to have a clean sequential index
associated_vars = associated_vars.reset_index(drop=True)

# Select only report-relevant columns
associated_vars = associated_vars[['Variable', 'Description', 'Chi2', 'p-value']]

# Display the filtered and cleaned table
from IPython.display import display
display(associated_vars)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from labels import translated_labels, variable_labels, map_codes, translate

# Outcome variable
outcome_var = 'D05'

# Predictor variables
predictors = ['D01', 'D02', 'D03', 'D04', 'D06_1', 'D07', 'D09',
              'D13', 'D15', 'D19', 'D20', 'regione']

# Translate dataframe for human-readable labels
df_plot = translate(df[[outcome_var] + predictors])

# Function to get ordered categories from translated_labels
def get_order(var):
    if var in translated_labels:
        sorted_items = sorted(translated_labels[var].items())
        return [v for k, v in sorted_items]
    else:
        # For non-Likert variables, just return sorted unique values
        return sorted(df_plot[var].dropna().unique())

# Define logical colour palette for D05 (Likert scale)
likert_palette = {
    'Strongly agree': '#2ca02c',  # vivid green
    'Agree': '#98df8a',           # light green
    'Neutral': '#d3d3d3',         # light grey
    'Disagree': '#c5b0d5',        # light purple
    'Strongly disagree': '#9467bd' # strong purple
}

# Loop over predictor variables
for var in predictors:
    # Get proper category order
    outcome_order = get_order(outcome_var)
    predictor_order = get_order(var)

    # Convert to ordered categorical
    df_plot[outcome_var] = pd.Categorical(df_plot[outcome_var],
                                          categories=outcome_order, ordered=True)
    df_plot[var] = pd.Categorical(df_plot[var],
                                  categories=predictor_order, ordered=True)

    # Build contingency table (rows=predictor, columns=outcome)
    counts = pd.crosstab(df_plot[var], df_plot[outcome_var])

    # Compute percentages
    counts_percent = counts.div(counts.sum(axis=1), axis=0) * 100

    # Plot stacked bar
    ax = counts_percent.plot(
        kind='bar', stacked=True, figsize=(10,6),
        color=[likert_palette.get(c, '#cccccc') for c in counts_percent.columns]  # fallback grey
    )

    # Add percentages inside bars
    for i, row in enumerate(counts_percent.values):
        cum_sum = 0
        for j, val in enumerate(row):
            if val > 0:
                ax.text(
                    i, cum_sum + val/2,
                    f"{val:.1f}%", 
                    ha='center', va='center', weight='bold', fontsize=10
                )
                cum_sum += val

    # Titles and axis labels using variable_labels
    ax.set_title(f"Distribution of {variable_labels.get(outcome_var)} by {variable_labels.get(var)}",
                 weight='bold', fontsize=12)
    ax.set_xlabel(variable_labels.get(var), weight='bold')
    ax.set_ylabel("Proportion (%)", weight='bold')
    ax.set_ylim(0, 100)
    plt.xticks(rotation=45, weight='bold')

    # Move legend to the right
    ax.legend(
        title=variable_labels.get(outcome_var),
        title_fontsize=10,
        fontsize=10,
        bbox_to_anchor=(1.02, 1),  # outside right
        loc='upper left',
        borderaxespad=0
    )

    plt.tight_layout()
    plt.show()


In [ ]:
# Plot distribution of D05 across categories of selected predictors
import matplotlib.pyplot as plt
import pandas as pd
from labels import translated_labels, variable_labels, map_codes, translate  # required labels file

# Custom ordinal color gradient for D05 categories:
# Adjust hex colors if needed, but these follow your semantics
d05_colors = {
    "Strongly agree": "#00A600",   # vivid green
    "Agree": "#8CD17D",            # soft light green
    "Neutral": "#D9D9D9",          # light grey
    "Disagree": "#CAB2D6",         # light purple
    "Strongly disagree": "#6A3D9A" # strong purple
}

d05_label = variable_labels.get('D05', 'D05')

predictors = ['D01', 'D02', 'D03', 'D04', 'D06_1', 'D07', 'D09',
              'D13', 'D15', 'D19', 'D20', 'regione']

figsize = (10, 6)
label_threshold_pct = 2.0

for var in predictors:

    if var not in df.columns:
        print(f"Warning: {var} not found in dataframe. Skipping.")
        continue

    subset = df[[var, 'D05']].dropna()

    # Apply label translation
    sub_disp = subset.copy()
    sub_disp[var] = map_codes(sub_disp[var], var)
    sub_disp['D05'] = map_codes(sub_disp['D05'], 'D05')

    # Build contingency table
    counts = pd.crosstab(sub_disp[var], sub_disp['D05'])

    if counts.empty:
        continue

    # Percent distribution
    counts_pct = counts.div(counts.sum(axis=1), axis=0) * 100

    # Order D05 responses according to the keys in translated_labels['D05']
    if 'D05' in translated_labels:
        ordered_cols = []
        for k in translated_labels['D05'].keys():
            lbl = translated_labels['D05'][k]
            if lbl in counts_pct.columns:
                ordered_cols.append(lbl)
        # Add missing ones if any
        for c in counts_pct.columns:
            if c not in ordered_cols:
                ordered_cols.append(c)

        counts_pct = counts_pct[ordered_cols]

    # Apply color map in correct order
    color_list = [d05_colors[c] for c in counts_pct.columns]

    # Plot
    ax = counts_pct.plot(
        kind="bar", 
        stacked=True, 
        figsize=figsize,
        color=color_list
    )

    # Label percentages inside segments
    for i, row in enumerate(counts_pct.values):
        cum = 0
        for j, val in enumerate(row):
            if val >= label_threshold_pct:
                ax.text(
                    i,
                    cum + val/2,
                    f"{val:.1f}%",
                    ha="center",
                    va="center",
                    color="white",
                    fontsize=10,
                    fontweight='bold'
                )
            cum += val

    # Titles and axes
    var_label = variable_labels.get(var, var)
    # plt.title(f"Distribution of {d05_label} by {var_label}")
    plt.title(f"“Attitude toward environmental policy” by {var_label}")
    plt.xlabel(var_label)
    plt.ylabel("Proportion (%)")
    plt.ylim(0, 100)
    plt.xticks(rotation=45, ha='right')

    plt.legend(title=d05_label, bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
# GLOBAL CRAMER'S V HEATMAP (all categorical variables × all categorical variables)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from labels import translated_labels, variable_labels, map_codes, translate

# Function to compute Cramér's V for two categorical variables
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.shape[0] < 2 or confusion_matrix.shape[1] < 2:
        return np.nan  # Skip degenerate tables
    chi2_stat, p, dof, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    return np.sqrt(chi2_stat / (n * (min(confusion_matrix.shape) - 1)))

# Compute a symmetric Cramér's V matrix for a list of categorical variables
def cramers_v_matrix(df, categorical_vars, variable_names=None):
    n = len(categorical_vars)
    matrix = pd.DataFrame(np.zeros((n, n)), index=categorical_vars, columns=categorical_vars)

    for i, var1 in enumerate(categorical_vars):
        for j, var2 in enumerate(categorical_vars):
            if i <= j:  # compute upper triangle
                v = cramers_v(df[var1], df[var2])
                matrix.loc[var1, var2] = v
                matrix.loc[var2, var1] = v  # symmetric

    # Optionally rename variables using variable_labels
    if variable_names:
        matrix.rename(index=variable_names, columns=variable_names, inplace=True)

    return matrix

# Plot the heatmap with discrete legend
def plot_cramers_v_heatmap_discrete(df, categorical_vars, variable_names=None, figsize=(14,12)):
    # Translate dataframe to readable labels
    df_translated = translate(df[categorical_vars])
    
    # Compute Cramér's V matrix
    cv_matrix = cramers_v_matrix(df_translated, categorical_vars, variable_names=variable_names)
    
    # Define discrete categories
    bins = [0.0, 0.10, 0.20, 0.40, 0.60, 0.80, 1.0]
    labels = [
        "Negligible / Very weak",
        "Weak",
        "Moderate",
        "Relatively strong",
        "Strong",
        "Very strong / Almost perfect"
    ]
    
    # Map values to discrete categories
    cv_cat = np.digitize(cv_matrix.fillna(0).values, bins, right=False) - 1
    cv_cat = np.clip(cv_cat, 0, len(labels)-1)
    
    # Discrete colormap: light to strong red
    colors = sns.light_palette("red", n_colors=len(labels), reverse=False)
    cmap = ListedColormap(colors)
    
    # Plot heatmap
    plt.figure(figsize=figsize)
    sns.heatmap(cv_cat, annot=np.round(cv_matrix.values,2), fmt=".2f",
                cmap=cmap, cbar=False, square=True,
                xticklabels=cv_matrix.columns, yticklabels=cv_matrix.index)
    
    # Create discrete legend
    legend_elements = [Patch(facecolor=cmap(i), edgecolor='k', label=labels[i]) for i in range(len(labels))]
    plt.legend(handles=legend_elements, title="Association Strength", bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=10)
    plt.title("Cramér's V Heatmap for Categorical Variables", fontsize=16, weight='bold')
    plt.tight_layout()
    plt.show()
    
    return cv_matrix

# List of categorical variables
categorical_vars = [
    "GEN", "RETA", "regione", 'D01', 'D02',
    'D03', 'D04', 'D05', 'D06_1', 'D07', 'D08', 'D09',
    'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D22'
]

# Compute and plot
cv_matrix = plot_cramers_v_heatmap_discrete(df, categorical_vars, variable_names=variable_labels)


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(x, y):
    """Calculate Cramér's V between two categorical vectors."""
    table = pd.crosstab(x, y)
    chi2 = chi2_contingency(table, correction=False)[0]
    n = table.sum().sum()
    phi2 = chi2 / n
    r, k = table.shape
    return np.sqrt(phi2 / (min(k - 1, r - 1)))

# CREATE THE MATRIX
cramers_v_df = pd.DataFrame(index=categorical_vars, columns=categorical_vars, dtype=float)

for row in categorical_vars:
    for col in categorical_vars:
        cramers_v_df.loc[row, col] = cramers_v(df[row], df[col])

cramers_v_df

def extract_strong_associations(cramers_v_df, threshold=0.20):
    results = []

    for row in cramers_v_df.index:
        for col in cramers_v_df.columns:
            if row != col:  # skip diagonal
                v = cramers_v_df.loc[row, col]
                if v >= threshold:
                    results.append((row, col, v))

    assoc_df = pd.DataFrame(results, columns=["Variable 1", "Variable 2", "Cramér’s V"])
    assoc_df = assoc_df.sort_values(by="Cramér’s V", ascending=False)

    # remove duplicate (A-B and B-A pairs)
    assoc_df = assoc_df[assoc_df["Variable 1"] < assoc_df["Variable 2"]]

    return assoc_df

strong_associations = extract_strong_associations(cramers_v_df, threshold=0.20)
strong_associations



In [ ]:
# 1. Convert Cramer matrix to long format
assoc = cramers_v_df.stack().reset_index()
assoc.columns = ["Var1", "Var2", "CramersV"]

# 2. Remove self-correlations and duplicates
assoc = assoc[assoc["Var1"] < assoc["Var2"]]

# 3. Keep only meaningful associations > 0.20
assoc = assoc[assoc["CramersV"] > 0.20]

# 4. Add variable descriptions
assoc["Var1 label"] = assoc["Var1"].map(variable_names)
assoc["Var2 label"] = assoc["Var2"].map(variable_names)

# 5. Categorize strength levels
def cramers_category(v):
    if 0.20 <= v < 0.40: return "Moderate"
    if 0.40 <= v < 0.60: return "Relatively strong"
    if 0.60 <= v < 0.80: return "Strong"
    if v >= 0.80:        return "Very strong"

assoc["Strength"] = assoc["CramersV"].apply(cramers_category)

# 6. Sort by highest association
assoc = assoc.sort_values(by="CramersV", ascending=False)

# 7. Apply color palette (BuPu shades)
def highlight_palette(val):
    colors = {
        "Moderate": "#c7e9f1",
        "Relatively strong": "#7eb7d6",
        "Strong": "#4675b4",
        "Very strong": "#2b3c8a"
    }
    return f"background-color: {colors.get(val, 'white')}; color: black"

# 8. Display formatted table
styled_table = assoc.style.applymap(highlight_palette, subset=["Strength"])
styled_table


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch

# 1. Compute Cramér's V
def cramers_v(x, y):
    table = pd.crosstab(x, y)
    chi2 = chi2_contingency(table, correction=False)[0]
    n = table.sum().sum()
    phi2 = chi2 / n
    r, k = table.shape
    return np.sqrt(phi2 / (min(k - 1, r - 1)))

# 2. Define your survey blocks
env_awareness = ["D01", "D02", "D03", "D04", "D05"]
env_behavior = ["D06_1", "D07", "D08", "D09", "D13"]
civic_engagement = ["D14", "D15", "D16", "D17", "D18", "D19", "D20"]
demographics = ["GEN", "RETA", "regione", "D22"]

blocks = {
    "Environmental Awareness and Regulatory Attitudes": env_awareness,
    "Environmental Behaviors": env_behavior,
    "Civic Engagement": civic_engagement,
    "Demographics": demographics,
}

# 3. Define thresholds and labels (as requested)
thresholds = [0.0, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = [
    "Negligible [0–0.10[",
    "Weak [0.10–0.20[",
    "Moderate [0.20–0.40[",
    "Relatively strong [0.40–0.60[",
    "Strong [0.60–0.80[",
    "Very strong [0.80–1.00]"
]

# BuPu color palette (6 categories)
palette = sns.color_palette("BuPu", len(labels))
cmap = ListedColormap(palette)
norm = BoundaryNorm(thresholds, cmap.N)

# 4. Compute block-level Cramér matrices
def compute_cramers_matrix(df, variables):
    matrix = pd.DataFrame(index=variables, columns=variables, dtype=float)
    for v1 in variables:
        for v2 in variables:
            matrix.loc[v1, v2] = 1.0 if v1 == v2 else cramers_v(df[v1], df[v2])
    return matrix

# 5. Plot block heatmap (NO continuous legend)
def plot_block_heatmap(matrix, title):
    plt.figure(figsize=(7, 6))

    sns.heatmap(
        matrix,
        annot=True,
        fmt=".2f",
        cmap=cmap,
        norm=norm,
        square=True,
        cbar=False,            # No vertical colorbar
        annot_kws={"size": 9}
    )

    plt.title(title, fontsize=14, fontweight="bold")

    # Discrete category legend only, no color bar
    legend_patches = [
        Patch(facecolor=palette[i], label=labels[i])
        for i in range(len(labels))
    ]
    plt.legend(
        handles=legend_patches,
        title="Association strength",
        bbox_to_anchor=(1.05, 1),
        loc="upper left"
    )

    plt.tight_layout()
    plt.show()

# 6. Generate heatmaps by thematic block
for block_name, vars_list in blocks.items():
    matrix = compute_cramers_matrix(df, vars_list)
    plot_block_heatmap(matrix, f"Cramér’s V — {block_name}")
